# 02-2 스크래핑 사용하기

In [ ]:
# 02-2 스크래핑 사용하기_김기중_2023183273

## (1) 전체 도서의 무게 구하기

In [94]:
def get_book_weight(isbn):
    # Yes24 도서 검색 페이지 URL
    url = 'http://www.yes24.com/Product/Search?domain=BOOK&query={}'
    # URL에 ISBN을 넣어 HTML 가져옵니다.
    r = requests.get(url.format(isbn))
    soup = BeautifulSoup(r.text, 'html.parser')   # HTML 파싱
    # 검색 결과에서 해당 도서를 선택합니다.
    prd_info = soup.find('a', attrs={'class':'gd_name'})
    if prd_info == None:
        return ''
    # 도서 상세 페이지를 가져옵니다.
    url = 'http://www.yes24.com'+prd_info['href']
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')
    # 상품 상세정보 div를 선택합니다.
    prd_detail = soup.find('div', attrs={'id':'infoset_specific'})
    # 테이블에 있는 tr 태그를 가져옵니다.
    prd_tr_list = prd_detail.find_all('tr')
    # 쪽수가 들어 있는 th를 찾아 td에 담긴 값을 반환합니다.
    for tr in prd_tr_list:
        if tr.find('th').get_text() == '쪽수, 무게, 크기':
            text_arr = tr.find('td').get_text().split()
            return text_arr[2]
    return ''

In [95]:
get_book_weight(9791190090018)

'496g'

In [96]:
book_weight = top10_books.apply(lambda row: get_book_weight(row['isbn13']), axis=1)
book_weight

0    496g
1    358g
2    296g
3    412g
4    388g
5    512g
6    318g
7        
8    324g
9    406g
dtype: object

In [99]:
book_weight.name = 'book_weight'
top10_with_book_weight = pd.merge(top10_books, book_weight,
                                 left_index=True, right_index=True)
top10_with_book_weight

,no,ranking,bookname,authors,publisher,publication_year,isbn13,book_weight
0,1,1,우리가 빛의 속도로 갈 수 없다면 :김초엽 소설,지은이: 김초엽,허블,2019,9791190090018,496g
1,2,2,달러구트 꿈 백화점.이미예 장편소설,지은이: 이미예,팩토리나인,2020,9791165341909,358g
2,3,3,지구에서 한아뿐 :정세랑 장편소설,지은이: 정세랑,난다,2019,9791188862290,296g
3,4,4,"시선으로부터, :정세랑 장편소설",지은이: 정세랑,문학동네,2020,9788954672214,412g
4,5,5,아몬드 :손원평 장편소설,지은이: 손원평,창비,2017,9788936434267,388g
5,6,6,피프티 피플 :정세랑 장편소설,지은이: 정세랑,창비,2016,9788936434243,512g
6,7,7,목소리를 드릴게요 :정세랑 소설집,지은이: 정세랑,아작,2020,9791165300005,318g
7,8,8,나미야 잡화점의 기적 :히가시노 게이고 장편소설,지은이: 히가시노 게이고 ;옮긴이: 양윤옥,현대문학,2012,9788972756194,
8,9,9,선량한 차별주의자,김지혜 지음,창비,2019,9788936477196,324g
9,10,9,쇼코의 미소 :최은영 소설,지은이: 최은영,문학동네,2016,9788954641630,406g


## (2) 전체 도서의 정보 구하기 (여러 필드 동시 추출)

In [72]:
def get_page_info(isbn):
    # Yes24 도서 검색 페이지 URL
    url = 'http://www.yes24.com/Product/Search?domain=BOOK&query={}'
    # URL에 ISBN을 넣어 HTML 가져옵니다.
    r = requests.get(url.format(isbn))
    soup = BeautifulSoup(r.text, 'html.parser')   # HTML 파싱
    # 검색 결과에서 해당 도서를 선택합니다.
    prd_info = soup.find('a', attrs={'class':'gd_name'})
    if prd_info == None:
        return ''
    # 도서 상세 페이지를 가져옵니다.
    url = 'http://www.yes24.com'+prd_info['href']
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')
    # 상품 상세정보 div를 선택합니다.
    prd_detail = soup.find('div', attrs={'id':'infoset_specific'})
    # 테이블에 있는 tr 태그를 가져옵니다.
    prd_tr_list = prd_detail.find_all('tr')
    # 쪽수가 들어 있는 th를 찾아 td에 담긴 값을 반환합니다.
    for tr in prd_tr_list:
        if tr.find('th').get_text() == '쪽수, 무게, 크기':
            text_arr = tr.find('td').get_text().split()
            return text_arr[0],text_arr[2],text_arr[4]
    return '','',''

In [55]:
get_page_info(9791190090018)

('330쪽', '496g', '130*198*30mm')

In [81]:
book_info = top10_books.apply(lambda row: get_page_info(row['isbn13']), axis=1).tolist()
book_info

[('330쪽', '496g', '130*198*30mm'),
 ('300쪽', '358g', '134*200*20mm'),
 ('228쪽', '296g', '130*195*15mm'),
 ('340쪽', '412g', '133*200*23mm'),
 ('264쪽', '388g', '134*195*20mm'),
 ('396쪽', '512g', '145*210*30mm'),
 ('272쪽', '318g', '137*197*18mm'),
 '',
 ('244쪽', '324g', '140*210*16mm'),
 ('296쪽', '406g', '145*210*20mm')]

In [80]:
df = pd.DataFrame(book_info, columns=['page_count', 'book_weight', 'book_size'])
df

,page_count,book_weight,book_size
0,330쪽,496g,130*198*30mm
1,300쪽,358g,134*200*20mm
2,228쪽,296g,130*195*15mm
3,340쪽,412g,133*200*23mm
4,264쪽,388g,134*195*20mm
5,396쪽,512g,145*210*30mm
6,272쪽,318g,137*197*18mm
7,None,None,None
8,244쪽,324g,140*210*16mm
9,296쪽,406g,145*210*20mm


In [92]:
top10_with_page_info = pd.merge(top10_books, df,
                                 left_index=True, right_index=True)
top10_with_page_info

,no,ranking,bookname,authors,publisher,publication_year,isbn13,page_count,book_weight,book_size
0,1,1,우리가 빛의 속도로 갈 수 없다면 :김초엽 소설,지은이: 김초엽,허블,2019,9791190090018,330쪽,496g,130*198*30mm
1,2,2,달러구트 꿈 백화점.이미예 장편소설,지은이: 이미예,팩토리나인,2020,9791165341909,300쪽,358g,134*200*20mm
2,3,3,지구에서 한아뿐 :정세랑 장편소설,지은이: 정세랑,난다,2019,9791188862290,228쪽,296g,130*195*15mm
3,4,4,"시선으로부터, :정세랑 장편소설",지은이: 정세랑,문학동네,2020,9788954672214,340쪽,412g,133*200*23mm
4,5,5,아몬드 :손원평 장편소설,지은이: 손원평,창비,2017,9788936434267,264쪽,388g,134*195*20mm
5,6,6,피프티 피플 :정세랑 장편소설,지은이: 정세랑,창비,2016,9788936434243,396쪽,512g,145*210*30mm
6,7,7,목소리를 드릴게요 :정세랑 소설집,지은이: 정세랑,아작,2020,9791165300005,272쪽,318g,137*197*18mm
7,8,8,나미야 잡화점의 기적 :히가시노 게이고 장편소설,지은이: 히가시노 게이고 ;옮긴이: 양윤옥,현대문학,2012,9788972756194,None,None,None
8,9,9,선량한 차별주의자,김지혜 지음,창비,2019,9788936477196,244쪽,324g,140*210*16mm
9,10,9,쇼코의 미소 :최은영 소설,지은이: 최은영,문학동네,2016,9788954641630,296쪽,406g,145*210*20mm
